In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras import utils

from tensorflow import image

from numpy.random import seed
seed(2020)
from tensorflow.random import set_seed
set_seed(2020)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dl-urfu-cifar10/train.npy
/kaggle/input/dl-urfu-cifar10/train_label.npy
/kaggle/input/dl-urfu-cifar10/sample_submission.csv
/kaggle/input/dl-urfu-cifar10/test.npy


In [2]:
# Данные хранятся в формате многомерного массива numpy
x_train = np.load('/kaggle/input/dl-urfu-cifar10/train.npy')
y_train = np.load('/kaggle/input/dl-urfu-cifar10/train_label.npy')
x_test = np.load('/kaggle/input/dl-urfu-cifar10/test.npy')

In [3]:
x_train.shape, y_train.shape, x_test.shape

((56000, 32, 32, 3), (56000, 1), (4000, 32, 32, 3))

In [4]:
# Нормализация данных
x_train = x_train / 255
x_test = x_test / 255

In [5]:
# One hot encoding для целевой переменной [0,0,0,0,0,...0]
y_train = utils.to_categorical(y_train, 10)

In [6]:
seed(2020)
set_seed(2020)

model = Sequential()
# Архитектура LeNet5
# Первый сверточный слой
model.add(Conv2D(6, (5,5), padding='same', input_shape=(32,32, 3), activation='relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2,2)))
# Второй сверточный слой
model.add(Conv2D(16, (5,5), activation='relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2,2)))
# Третий сверточный слой
model.add(Conv2D(120, (5,5), activation='relu'))
model.add(Flatten())
# Полносвязный слой
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))




In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 6)         456       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 120)         48120     
_________________________________________________________________
flatten (Flatten)            (None, 480)               0         
_________________________________________________________________
dense (Dense)                (None, 84)                4

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [9]:
model.fit(x_train, y_train, batch_size=5, epochs=20, validation_split=0.1, verbose=2)

Epoch 1/20
10080/10080 - 65s - loss: 1.5594 - accuracy: 0.4280 - val_loss: 1.3769 - val_accuracy: 0.5121
Epoch 2/20
10080/10080 - 64s - loss: 1.2942 - accuracy: 0.5382 - val_loss: 1.2190 - val_accuracy: 0.5663
Epoch 3/20
10080/10080 - 64s - loss: 1.1775 - accuracy: 0.5830 - val_loss: 1.2415 - val_accuracy: 0.5636
Epoch 4/20
10080/10080 - 64s - loss: 1.0961 - accuracy: 0.6125 - val_loss: 1.1870 - val_accuracy: 0.5923
Epoch 5/20
10080/10080 - 64s - loss: 1.0368 - accuracy: 0.6343 - val_loss: 1.1399 - val_accuracy: 0.6023
Epoch 6/20
10080/10080 - 64s - loss: 0.9855 - accuracy: 0.6525 - val_loss: 1.1699 - val_accuracy: 0.6066
Epoch 7/20
10080/10080 - 64s - loss: 0.9449 - accuracy: 0.6669 - val_loss: 1.1155 - val_accuracy: 0.6261
Epoch 8/20
10080/10080 - 64s - loss: 0.9103 - accuracy: 0.6787 - val_loss: 1.1681 - val_accuracy: 0.6150
Epoch 9/20
10080/10080 - 64s - loss: 0.8807 - accuracy: 0.6893 - val_loss: 1.1655 - val_accuracy: 0.6080
Epoch 10/20
10080/10080 - 64s - loss: 0.8516 - accuracy

In [10]:
predictions = model.predict(x_test)
#  Извлекаем номера предсказаний с максимальными вероятностями по всем объектам тестового набора
predictions = np.argmax(predictions, axis=1)

In [11]:
sample_submission = pd.read_csv('/kaggle/input/dl-urfu-cifar10/sample_submission.csv')
sample_submission['label'] = predictions
sample_submission.to_csv('sample_submission.csv', index=False)